In [44]:
import pandas as pd
import numpy as np
import re
import primary_f

In [45]:
# Lectura del CSV y guardado en una variable
def READ_CSV(x):
    return pd.read_csv(x, sep=",", encoding='latin-1')

FILE = READ_CSV('altas_bajas_3.csv')

# Guardo los emails en una sola colimna de un DF
FILE_DF = pd.DataFrame()

In [46]:
#Indentify of the column name cotain this regular expresion: "ID"
def id_list(x):
    column_list = x.columns
    patron = re.compile(r"ID", re.IGNORECASE)
    resultados = []   
    
    for y in column_list:
        if re.search(patron, str(y)): 
           resultados.append(True)
        else:
            resultados.append(False)
            
    return(resultados)

In [47]:
def id_probability(x):
    id_re = [0 if is_id else 1 for is_id in id_list(x)]
    count_rows = primary_f.r_count(x)
    count_distinct_rows = primary_f.r_count_distinct(x)

    unique_and_distincts = [1 if x1 != x2 else 0 for x1, x2 in zip(count_rows, count_distinct_rows)]
    empty_rows = [1 if x > 0 else 0 for x in primary_f.r_empty(x)]
    null_rows = [1 if x > 0 else 0 for x in primary_f.is_null(x)]

    # Imprimir longitudes para depuración
    print(f"Lengths - id_re: {len(id_re)}, unique_and_distincts: {len(unique_and_distincts)}, empty_rows: {len(empty_rows)}, null_rows: {len(null_rows)}")

    lengths = [len(id_re), len(unique_and_distincts), len(empty_rows), len(null_rows)]
    if len(set(lengths)) != 1:
        raise ValueError(f"Mismatch in lengths: {lengths}")

    percent_unique_1 = [(x2 / x1 * 100) if x1 > 0 else 0 for x1, x2 in zip(count_rows, count_distinct_rows)]

    percent_unique_2 = []
    for x in percent_unique_1:
        if x >= 90.1:
            percent_unique_2.append(0)
        elif x >= 50.1:
            percent_unique_2.append(1)
        elif x >= 20.1:
            percent_unique_2.append(2)
        else:
            percent_unique_2.append(3)

    final = pd.DataFrame({
        'id_re': id_re,
        'unique_and_distincts': unique_and_distincts,
        'empty_rows': empty_rows,
        'null_rows': null_rows,
        'percent_unique': percent_unique_2
    })

    final_2 = final.sum(axis=1)

    id_quality = []
    for x in final_2:
        if x == 0:
            id_quality.append('100%')
        elif x == 1:
            id_quality.append('75%')
        elif x == 2:
            id_quality.append('50%')
        else:
            id_quality.append('0%')

    return id_quality



In [48]:
print(id_probability(FILE))

Lengths - id_re: 83, unique_and_distincts: 82, empty_rows: 83, null_rows: 83


ValueError: Mismatch in lengths: [83, 82, 83, 83]